In [125]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from matplotlib import style
import datetime
import seaborn as sns

## This Notebook combines all the county data 

In [126]:
###PA.gov
def yn_switch(opdfnew):
    opdfnew["Narcan Admin"] = opdfnew['Naloxone Administered'].map(lambda x: 1 if x == "Y" else 0)
    opdfnew["Survive"] = opdfnew['Survive'].map(lambda x: 1 if x == "Y" else 0)

    return(opdfnew)   
df = pd.read_csv('../data/PAGOV.csv')
df.head()
opiod_list = ['CARFENTANIL','FENTANYL','FENTANYL ANALOG/OTHER SYNTHETIC OPIOID','HEROIN','METHADONE','PHARMACEUTICAL OPIOID','SUBOXONE']

# Reduce incidents to ones with specific opioid
opdf = df.loc[df['Susp OD Drug Desc'].isin(opiod_list)]

# Store relevant information
opdf = opdf[['Incident ID','Incident County Name','Incident Date','Victim ID','Gender Desc','Day','Naloxone Administered','Age Range','Survive','Response Desc','Incident Time']]

# Remove duplicates
opdf = opdf.drop_duplicates()
opdf['Incident Date ym'] = pd.to_datetime(opdf['Incident Date']).dt.to_period('Y')

opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(str)
opdf['Incident Date ym'] = opdf['Incident Date ym'].astype(int)

oppdf = opdf.groupby(['Incident Date ym'])[['Incident ID']].count().reset_index()
opdfnew = yn_switch(opdf.copy())

#limitign
opdfnew = opdfnew[['Incident ID','Incident County Name',
                   'Narcan Admin','Survive','Incident Date ym']]

#Renaming for joins later
opdfnew  = opdfnew.rename(columns={'Incident County Name': 'County','Incident Date ym':'Year'})

##Getting totals

opdfnew['Total Overdoses Per County'] = opdfnew.groupby(['County','Year'])['County'].transform('count')

opdfnew['Percent Narcan Admin Per County/Year'] = opdfnew.groupby(['County','Year'])['Narcan Admin'].transform('mean')
opdfnew['Percent Survive Overdose Per County/Year'] = opdfnew.groupby(['County','Year'])['Survive'].transform('mean')

##Limiting

opdfnew = opdfnew[['County','Year','Percent Narcan Admin Per County/Year','Percent Survive Overdose Per County/Year']]

opdfnew = opdfnew.drop_duplicates()


In [127]:
opdfnew.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year
2,Delaware,2018,0.656250,0.703125
4,Chester,2018,0.813397,0.751196
5,Beaver,2018,0.685185,0.814815
6,Bucks,2018,0.739895,0.804921
8,Philadelphia,2018,0.951220,0.926829


In [128]:
### Next, Takeback and treaments
takebackdf = pd.read_csv('../data/Prescription_Drug_Take-Back_Box_Locations_County_Drug_and_Alcohol_Programs.csv')
### There is some datacleanup needed as some counties are in all caps and listed as PA

def fix(df):
    for i, row in df.iterrows():
        try:
            cngcnty = row['County'].split(',')[0]
            cngcnty = str.title(cngcnty)
            df.at[i,row['County']] = cngcnty
        except:
            pass
    return(df)        
takebackdf1 = fix(takebackdf.copy())

takebackdf1['Total of Take Back Locations by County'] = takebackdf1.groupby('County')['County'].transform('count')




###Loading treatment
treatdf = pd.read_csv('../data/Drug_and_Alcohol_Treatment_Facilities_May_2018_County_Drug_and_Alcohol_Programs.csv')

treatdf['Total of Treatment Locations by County'] = treatdf.groupby('County')['County'].transform('count')

treatdf = treatdf[['County','Total of Treatment Locations by County']]

### Merging dataframes
dfcnt2 = pd.merge(takebackdf1,treatdf, how="inner",left_on = 'County',right_on = 'County')

In [129]:
dftakebackandtreatment = dfcnt2[['County','Total of Take Back Locations by County','Total of Treatment Locations by County']]
dftakebackandtreatment = dftakebackandtreatment.drop_duplicates()

In [130]:
dfmerge1 = pd.merge(opdfnew,dftakebackandtreatment, how="inner",left_on = 'County',right_on = 'County')

In [131]:
dfmerge1.head()  ##PA.gov and takebacks/treatments

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County
0,Delaware,2018,0.656250,0.703125,55,30
1,Delaware,2019,0.717949,0.615385,55,30
2,Delaware,2020,0.555556,0.587302,55,30
3,Delaware,2021,0.560000,0.700000,55,30
4,Delaware,2022,0.500000,0.714286,55,30


In [132]:
###Risky Precribing dataset
df = pd.read_csv('../data/Risky_Prescribing_Measures_Quarter_3_2016_-_Current_Quarterly_County___Statewide_Health.csv')

### Removing PA as to not count twice
df = df[df['County'] != 'Pennsylvania']

##Suming per county/year

df['Total Risky Prescibing per County/Year'] = df.groupby(['County','Year'])['Rate or Count'].transform('sum')

#limiting
dfrisky = df[['County','Year','Total Risky Prescibing per County/Year']]




In [133]:
dfmerge1['Year'] = dfmerge1['Year'].astype(int)

In [134]:
dfrisky.head()

,County,Year,Total Risky Prescibing per County/Year
1,Lackawanna,2020,36468.040054
4,Mifflin,2021,3224.363865
5,Schuylkill,2021,11536.172907
6,Butler,2019,26160.090000
7,Snyder,2016,4034.040000


In [135]:
dfmerge1.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County
0,Delaware,2018,0.656250,0.703125,55,30
1,Delaware,2019,0.717949,0.615385,55,30
2,Delaware,2020,0.555556,0.587302,55,30
3,Delaware,2021,0.560000,0.700000,55,30
4,Delaware,2022,0.500000,0.714286,55,30


In [136]:
##Merging risky dataset
dfmerge2 = pd.merge(dfmerge1,dfrisky, how="inner",left_on = ['County','Year'],right_on = ['County','Year'])

In [137]:
dfmerge2.head()

,County,Year,Percent Narcan Admin Per County/Year,Percent Survive Overdose Per County/Year,Total of Take Back Locations by County,Total of Treatment Locations by County,Total Risky Prescibing per County/Year
0,Delaware,2018,0.65625,0.703125,55,30,87748.56
1,Delaware,2018,0.65625,0.703125,55,30,87748.56
2,Delaware,2018,0.65625,0.703125,55,30,87748.56
3,Delaware,2018,0.65625,0.703125,55,30,87748.56
4,Delaware,2018,0.65625,0.703125,55,30,87748.56


In [ ]:
### Then we can add in rest for main dashboard